# Berman Standard State Code Generator 
Required system packages and initialization

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
import copy
sym.init_printing()

Required ENKI packages

In [ ]:
from thermoengine import coder

# Types of terms in a standard state properties description
There are three classes of terms:
1. Terms that apply over the whole of $T$-, $P$-space, $T_r \le T$, $P_r \le P$
2. Terms that apply over a specified range of $T$-, $P$-space, $(T_{r_\lambda},P_{r_\lambda}) \le (T,P) \le (T_\lambda,P_\lambda)$
3. Terms that apply to a specific $T_t$ and $P_t$ and higher $T$, $P$, $T_t \le T$, $P_t \le P$

Second-order phase transitions ($lambda$-transitions) are an example of the second type, as are order disorder transformations. First-order phase transitions are an example of the third type.  

## Create a model class for the Gibbs free energy

In [ ]:
model = coder.StdStateModel()

Retrieve sympy symbols for model variables and reference conditions

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

### Define model expressions applicable over all of T,P space
An expression for the Gibbs free energy, $G(T,P)$ or the Helmholtz energy $A(T,V)$ is constructed.  The expression may have multiple parts.  Often the heat capacity function is postulated, then integrated to yield expressions for the entahlpy, entropy, and in combination the energy potential. Then, an equation of state (EOS) is adopted and that term is integrated in pressure or volume and added to the heat capacity integrals. This proceedure is follwed here.
#### (1) $C_P$ integrals
The isobaric heat capacity terms parameterized as: $C_P = k_0 + k_1 / T^{1/2} + k_2 / T^2 + k_3 / T^3 $,
and in addition the reference condition third law entropy, $ S_{Tr,Pr} $, and enthalpy of formation from the
elements, $ \Delta H_{Tr,Pr} $, constitute additional parameters:

In [ ]:
k0,k1,k2,k3 = sym.symbols('k0 k1 k2 k3')
CpPr = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3
STrPr,HTrPr = sym.symbols('S_TrPr H_TrPr')

In [ ]:
CpPr

Specify paramters ...

In [ ]:
params = [('H_TrPr','J',HTrPr), ('S_TrPr','J/K',STrPr), ('k0','J/K-m',k0), ('k1','J-K^(1/2)-m',k1),
          ('k2','J-K/m',k2),  ('k3','J-K^2',k3)]

Define the heat capacity contribution to the Gibbs free energy ...

In [ ]:
GPr = HTrPr + sym.integrate(CpPr,(T,Tr,T)) - T*(STrPr + sym.integrate(CpPr/T,(T,Tr,T)))

In [ ]:
GPr

... and add this expression to the model

In [ ]:
model.add_expression_to_model(GPr, params)

#### (2) $V$ (EOS) integrals
Next, define a volume-explicit equation of state applicable over the whole of temperature and pressure space

In [ ]:
VTrPr,v1,v2,v3,v4 = sym.symbols('V_TrPr v1 v2 v3 v4')
params = [('V_TrPr', 'J/bar-m', VTrPr), ('v1','1/bar',v1), ('v2','1/bar^2',v2), ('v3','1/K',v3),  ('v4','1/K^2',v4)]

In [ ]:
GPrToP = sym.integrate(VTrPr*(1+v1*(P-Pr)+v2*(P-Pr)**2+v3*(T-Tr)+v4*(T-Tr)**2),(P,Pr,P))
model.add_expression_to_model(GPrToP, params)

### Define additional lambda heat capacity terms applicable over a restricted range of T,P space
These contributions to the potential function apply over a limited range of $T,P$ or $T,V$ space.  However, the affects of these functions propagate beyond the upper limits of the range.  Say, $f(T,P)$ is the contribution to the Gibbs free energy that describes a $\lambda$-like transition over the range $({T_{\lambda ,ref}},{P_{\lambda ,ref}})$ to $({T_\lambda },{P_\lambda })$.  Then, *above the upper limit* of temperature there is a fixed entropy contribution:  
  
$- {\left. {\frac{{\partial f\left( {T,P} \right)}}{{\partial T}}} \right|_{{T_\lambda },{P_\lambda }}} = {f_S}({T_\lambda },{P_\lambda })$  
  
and *above the upper limit of pressure* there is a fixed volume contribution:  
  
${\left. {\frac{{\partial f\left( {T,P} \right)}}{{\partial P}}} \right|_{{T_\lambda },{P_\lambda }}} = {f_V}({T_\lambda },{P_\lambda })$  

the consequence of which is that for $T > T_{\lambda}$ and $P > P_{\lambda}$, there is a contribution to the Gibbs free energy of the form:  

$- \left( {T - {T_\lambda }} \right){f_S}({T_\lambda },{P_\lambda }) + \left( {P - {P_\lambda }} \right){f_V}({T_\lambda },{P_\lambda })$  

This contribution is linear in $T$ and $P$.  

The additional energetic contributions applicable above the upper range limit will be added automatically to the model function, and need not be explicitly accounted for in building the model expressions.  

#### (1) $\lambda$-transition-like heat capacity integrals 
Parameters of the Berman (1988) lambda transition model:
- $l_1$ and $l_2$, coefficients in Berman (1988)'s $lambda$-heat capacity model
- $k_{\lambda}$, $\frac{{d{T_\lambda }}}{{dP}}$ in Berman (1988)'s $lambda$-transition model
- $T_{\lambda,{P_r}}$, Temperature of the $\lambda$-transition at reference pressure
- $T_{\lambda,{ref}}$, Temperature of the lower bound of the heat capacity integral for the $\lambda$-transition at reference pressure

In [ ]:
l1,l2 = sym.symbols('l1 l2')
kl = sym.symbols('k_lambda')
TlPr, Tlref = sym.symbols('T_lambda_Pr T_lambda_ref')
params = [('l1','(J/m)^(1/2)-K', l1), ('l2', '(J/m)^(1/2)/K^2', l2), ('k_lambda', 'K/bar', kl), 
          ('T_lambda_Pr', 'K', TlPr), ('T_lambda_ref', 'K', Tlref)]

Calculate the transition temperature, $T_{\lambda}$, at the pressure, $P$

In [ ]:
Tl = TlPr + kl*(P-Pr)

Temperature difference between $T_{\lambda}$ at $P$ and $P_r$

In [ ]:
td = TlPr - Tl

Reference temperature for lower limit of heat capacity integral.

In [ ]:
tr = Tlref - td

Heat capacity due to the $\lambda$-transition at $T$ and $P$. Valid: $T_r \le T \le T_{\lambda}$.  

**Note:** The syntax of the arguments to the SymPy Piecewise expression is of the form:
```
(e1,c1), (e2,d2), ..., (eDefault, True)
```
where the sequence is evaluated left to right. *eN* is the value of the resulting expression if the condition *cN* is *True*.  *cN* is always a logical comparison.  The first *cN* that is *True* provides the value of the expression.  If no *cN* evaluate to *True*, then the resulting expression becomes *eDefault* given by the last tuple in the sequence.  See the example below.

In [ ]:
Cpl = (T+td)*(l1+l2*(T+td))**2

Compute the Gibbs free energy of the lambda transition

In [ ]:
Gl = sym.integrate(Cpl,(T,tr,T)) - T*sym.integrate(Cpl/T,(T,tr,T))

... and add this expression to the model

In [ ]:
model.add_expression_to_model(Gl, params, exp_type='restricted', lower_limits=(tr,Pr), upper_limits=(Tl,None))

#### (2) First order phase transition terms
Berman terms valid at T $\ge$ $T_t$. Parameters (in this case $T_t$ is equivalent to $T_{\lambda}$:
- ${{\Delta}_t}H$, First order enthalpy contribution at $T_{\lambda}$

In [ ]:
deltaHt = sym.symbols('H_t')
params = [('H_t','J/m', deltaHt)]

${{\Delta}_t}S = {{\Delta}_t}H/T_{\lambda}$, First order enropy contribution at $T_{\lambda}$: ${{\Delta}_t}H - T {{\Delta}_t}S = -(T-T_{\lambda}) {{\Delta}_t}H/T_{\lambda}$ 

In [ ]:
GaboveTl = -(T-Tl)*deltaHt/Tl

... and add this expression to the model

In [ ]:
model.add_expression_to_model(GaboveTl , params, exp_type='restricted', lower_limits=(Tl,None), upper_limits=(Tl,None))

#### (3) Order-disorder contributions
Parameters of the Berman (1988) order-disorder model:
- $d_0$, $d_1$, $d_2$, $d_3$, $d_4$, $d_5$, order-disorder coefficients from the Berman (1988) model
- $T_{D_{ref}}$, $T_D$, minimum, maximum temperature of ordering interval, $T_{D_{ref}} \le T \le T_D$

In [ ]:
d0,d1,d2,d3,d4,d5 = sym.symbols('d0 d1 d2 d3 d4 d5')
TD,TDref = sym.symbols('T_D T_D_ref')
params = [('d0','J/K-m', d0), ('d1','J/K^(1/2)-m',d1), ('d2','J-K/m',d2), ('d3','J/K^2-m',d3), 
          ('d4','J/K^3-m',d4), ('d5','bar',d5), ('T_D','K',TD), ('T_D_ref','K',TDref)]

In [ ]:
CpDs = d0 + d1/sym.sqrt(T) + d2/T**2 + d3*T + d4*T**2
HDs = sym.integrate(CpDs,(T,TDref,T))
SDs = sym.integrate(CpDs/T,(T,TDref,T))
VDs = HDs/d5
GDs = HDs - T*SDs + VDs*(P-Pr)

In [ ]:
model.add_expression_to_model(GDs , params, exp_type='restricted', lower_limits=(TDref,None), upper_limits=(TD,None))

## Code Print the Model, compile the code and link a Python module
Name the model class

In [ ]:
model.set_module_name('Garnet')
model.get_berman_std_state_database()

Make a working sub-directory and move down into the directory.  This is done so that generated files will not clash between alternate model configurations.

In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

- Choose an existing phase from the Berman database
- Generate an include file and code file for this phase  
  
Note that the call to
```
model.create_code_module(phase=phase_name, formula=formula, params=param_dict)
```
generates fast code with unmodifiable model parameters and "calibration-" related functions.  The call to:
```
model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type='calib')
```
generates code suitable for model parameter calibration.  
model_type is "fast" or "calib"

In [ ]:
model_type = "calib"

#### Index 34 in Berman database is Almandine

In [ ]:
param_dict = model.get_berman_std_state_database(34)
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)


In [ ]:
param_dict['H_TrPr']=-5265440.0

In [ ]:
-5265440.0+5267216.0

In [ ]:
param_dict['H_TrPr']=-5267216.0
param_dict['S_TrPr']=340.007
param_dict['k0']=573.96
param_dict['k1']=-1483.1
param_dict['k2']=-29292000
param_dict['k3']=5022080000
param_dict['V_TrPr']=11.511
param_dict['v1']=-0.558E-6
param_dict['v2']=0.321E-12
param_dict['v3']=18.613E-6
param_dict['v4']=74.539E-10

In [ ]:
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

In [ ]:
param_dict

#### Index 22 in Berman database is Grossular

In [ ]:
param_dict = model.get_berman_std_state_database(22)
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)

In [ ]:
# try adding 1800 J here

In [ ]:
param_dict['H_TrPr']=-6632859.0
param_dict['S_TrPr']=255.150
param_dict['k0']=573.43 
param_dict['k1']=-2039.4
param_dict['k2']=-18887200
param_dict['k3']=2319310000
param_dict['V_TrPr']=12.538
param_dict['v1']=-0.654E-6
param_dict['v2']=1.635E-12
param_dict['v3']=18.994E-6
param_dict['v4']=79.756E-10

In [ ]:
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

In [ ]:
param_dict

#### Index 47 in Berman database is Pyrope

In [ ]:
param_dict = model.get_berman_std_state_database(47)
phase_name = param_dict.pop('Phase', None).title()
formula = param_dict.pop('Formula', None)


In [ ]:
-6284740.0+6286548.0

In [ ]:
# BErman96
param_dict['H_TrPr']=-6284740.0
param_dict['S_TrPr']=268.8
param_dict['k0']=590.9
param_dict['k1']=-2827.0
param_dict['k2']=-13320800
param_dict['k3']=1260330000
param_dict['V_TrPr']=11.311
param_dict['v1']=-0.576E-6
param_dict['v2']=0.442E-12
param_dict['v3']=22.519E-6
param_dict['v4']=37.044E-10

In [ ]:
# Berman91
param_dict['H_TrPr']=-6286548.0
param_dict['S_TrPr']=266.359
param_dict['k0']=640.72 
param_dict['k1']=-4542.1
param_dict['k2']=-4701900
param_dict['k3']=0
param_dict['V_TrPr']=11.316
param_dict['v1']=-0.576E-6
param_dict['v2']=0.442E-12
param_dict['v3']=22.519E-6
param_dict['v4']=37.044E-10

In [ ]:
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

In [ ]:
param_dict

## Import the new module and test the model

In [ ]:
import Garnet
%cd ..